In [ ]:
import pandas as pd
import tkinter
from tkinter import ttk
import os
import tkinter.messagebox
import customtkinter
import unicodedata
import requests
import tempfile
import chardet
from CTkTable import *
from tkinter import filedialog
from PIL import Image, ImageTk
from tkintertable.Tables import TableCanvas
from pandasgui import show
from pandastable import Table, TableModel
from pandastable.data import TableModel
# from pandastable.plugins import DataMergePlugin
from simpledbf import Dbf5

# Definindo a aparência padrão como "light"
customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

class DataFrameTable(Table):
    def __init__(self, parent=None, dataframe=None, showtoolbar=True, showstatusbar=True, **kwargs):
        model = TableModel(dataframe)
        Table.__init__(self, parent, model=model, showtoolbar=showtoolbar, showstatusbar=showstatusbar, **kwargs, df_table_principal=None, df_sub_tabela=None)
        # Table.merge()
        # self.datapopup = self.datawidget.datatable.popup
        # self.datapopup.addAction(self.datapopup.separator)
        # self.datapopup.addAction('Data Merge', lambda: self.add_plugin(DataMergePlugin))

class AppSaudados(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # Definindo a aparência padrão como "light"
        customtkinter.set_appearance_mode("light")
        customtkinter.set_default_color_theme("green")

        self.title("SAUDADOS")
        self.geometry(f"{1100}x{580}")
        self.resizable(width=False, height=False)

        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((1, 2), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=10, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        #titulo
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="SAUDADOS – CIÊNCIA DE DADOS E SAÚDE", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        #entrada
        self.Entrada_label = customtkinter.CTkLabel(self.sidebar_frame, text="Olá, Bem Vindo!", font=customtkinter.CTkFont(size=10, weight="bold"))
        self.Entrada_label.grid(row=1, column=0, padx=20, pady=(10))

        texto = "O objetivo desse sistema é apoiar as ações de pesquisa\n\n" +"utilizando dados de bases da saúde, bem como dados\n\n" +"demográficos entre outros.\n\n" + "Instruções:\n\n"
        
        # Usando Label para exibir o texto
        self.labelTexto = customtkinter.CTkLabel(self.sidebar_frame, text=texto, justify="left", font=customtkinter.CTkFont(size=10))
        self.labelTexto.grid(row=2, column=0, padx=(20, 0), pady=(20, 0), sticky="nsew")

        # Cria um arquivo temporário para armazenar temporariamente o DataFrame
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)

        self.dataframes_carregados = []
        self.colunas_selecionadas_primeiro_arquivo = []
        self.colunas_selecionadas_segundo_arquivo = []

        # Adicione esta linha para inicializar a variável de cancelamento
        self._cancel_load = False
        

        #mudar tema
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Aparência:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                    command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="Escala:", anchor="w")
        self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
                                                            command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))
        
        self.selected_file = None  # Armazenar o arquivo selecionado
        self.selected_extension = None  # Armazenar o tipo de extensão selecionado

        #fundo  
        self.imagem_label = customtkinter.CTkFrame(self, width=200, corner_radius=0)
        self.imagem_label.grid(row=0, column=1, rowspan=5, padx=(10, 0), pady=(0, 0), sticky="nsew")
        self.imagem_label.grid_rowconfigure(4, weight=1)

    #    # Carregue a imagem usando ImageTk.PhotoImage
    #     self.ctk_image = ImageTk.PhotoImage(Image.open("./fundoSAUDADOS.jpg"))
    #     self.image_label = customtkinter.CTkLabel(self.imagem_label, image=self.ctk_image)
    #     self.image_label.grid(row=0, column=1, rowspan=5, sticky="nsew")

    #     # Atualize a imagem
    #     self.image_label.configure(image=self.ctk_image)

        #base de dados
        self.label_base_dados = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Base de Dados:", anchor="w")
        self.label_base_dados.grid(row=0, column=0, padx=(40, 0), pady=(10, 0))

        # Obter todos os arquivos de um diretório específico
        diretorio = "D:/Area de Trabalho/Repositorys/PIBITI-IFBA-CNPq-2023-2024/ProjetoSaudados/BaseDadosTeste-Saudados"  
        arquivos = [arq for arq in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arq))]

        self.menu_base_dados = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", *arquivos], command=self.on_base_dados_selected)
        self.menu_base_dados.grid(row=1, column=0, padx=(40, 0), pady=(10, 10))

        # EXTENSAO
        self.label_extensao = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Extensão:", anchor="w")
        self.label_extensao.grid(row=2, column=0, padx=(40, 0), pady=(10, 0))
        self.menu_extensao = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", ".csv", ".xlsx", ".json", ".dbf", ".txt"], command=self.on_extensao_selected)
        self.menu_extensao.grid(row=3, column=0, padx=(40, 0), pady=(10, 10))

        #ISO OU UTF-8
        self.label_codificacao = customtkinter.CTkLabel(self.imagem_label, text="Selecione o Tipo de Upload:", anchor="w")
        self.label_codificacao.grid(row=4, column=0, padx=(40, 0), pady=(10, 0))
        self.menu_codificacao = customtkinter.CTkOptionMenu(self.imagem_label, values=[" ", "iso", "utf-8"], command=self.on_codificacao_selected)
        self.menu_codificacao.grid(row=5, column=0, padx=(40, 0), pady=(10, 10))

        #Exclusão de acentos ou caracteres especiais;
        self.checkbox_caracteres = customtkinter.CTkCheckBox(self.imagem_label, text="Exclusão de acentos\n" + "ou caracteres especiais")
        self.checkbox_caracteres.grid(row=6, column=0, pady=(90,0), padx=(20,0), sticky="n")

        #Botao ler arquivo
        self.button_LerArquivo = customtkinter.CTkButton(self.imagem_label, text="Ler Arquivo", command=self.Ler_Arquivo_DataFrame)
        self.button_LerArquivo.grid(row=8, column=0, padx=(40, 0), pady=(10, 20))

        # Adicione este botão à interface
        self.button_cancel_load = customtkinter.CTkButton(self, text="Cancelar Carregamento", command=self.cancel_load)
        self.button_cancel_load.grid(row=5, column=1, padx=(0, 0), pady=(10, 10))

        # Debug
        self.label_debug = customtkinter.CTkFrame(self, width=50, corner_radius=0)
        self.label_debug.grid(row=6, column=1, columnspan=2, sticky="nsew")
        self.label_debug.grid_rowconfigure(4, weight=1)

        # Debug entry
        self.entry = customtkinter.CTkLabel(self.label_debug, text="Selecione um arquivo")
        self.entry.grid(row=6, column=1)

        self.progress_bar = tkinter.ttk.Progressbar(self.label_debug, orient='horizontal', length=500, mode='determinate')
        self.progress_bar.grid(row=7, column=1, padx=20, pady=(10, 10))

        # create tabview
        self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="Selecione as Colunas:")
        self.scrollable_frame.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.scrollable_frame.grid_columnconfigure(0, weight=1)
        self.scrollable_frame_switches = []

        #join
        self.checkbox_JOIN = customtkinter.CTkCheckBox(self, text="Realizar Join")
        self.checkbox_JOIN.grid(row=1, column=2, pady=(20, 0), padx=0, sticky="n")
        self.checkbox_JOIN.grid_remove()  # Inicia o checkbox_JOIN como oculto

        # Botão para carregar as colunas selecionadas
        self.button_carregar_colunas = customtkinter.CTkButton(self, text="Carregar DataFrame", command=self.carregar_colunas_selecionadas)
        self.button_carregar_colunas.grid(row=1, column=2, padx=20, pady=50)

        # Adicionando variáveis para armazenar os valores dos switches
        self.switch_variables = []

        # # Adiciona este botão à interface
        # self.button_cancel_load = customtkinter.CTkButton(self, text="Cancelar Carregamento", command=self.cancel_load)
        # self.button_cancel_load.grid(row=5, column=3, padx=(0, 0), pady=(10, 10))

    def dbf2DF(self, file_path):
        dbf = Dbf5(file_path)
        return dbf.to_dataframe()

    def change_appearance_mode_event(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)
    
    @staticmethod
    def remove_special_characters(text):
        text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
        return text.upper()
    
    def determinar_delimitador(self, arquivo):
        with open(arquivo, 'r') as file:
            line = file.readline()
            delimiters = [',', ';', '\t']
            for delimiter in delimiters:
                if delimiter in line:
                    return delimiter
            return ';'
    
    def on_base_dados_selected(self, value):
        self.selected_file = value  # Armazenar o arquivo selecionado ao escolher na opção
        self.entry.configure(text=f"Selecionado o arquivo {self.selected_file}")

    def on_extensao_selected(self, value):
        self.selected_extension = value  # Armazenar a extensão selecionada ao escolher na opção
        self.entry.configure(text=f"Selecionada a extensão {self.selected_extension}")

    def on_codificacao_selected(self, value):
        self.selected_codificacao = value
        self.entry.configure(text=f"Selecionada a codificação {self.selected_codificacao}")

    def mostrar_colunas_disponiveis(self, df):
        columns = df.columns.tolist()  # Obtém os nomes das colunas como uma lista
        # Após a leitura bem-sucedida do arquivo CSV, cria os interruptores para as colunas
        for i, col in enumerate(columns):
            switch_var = tkinter.IntVar(value=0)  # Variável para armazenar o valor do interruptor
            switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"{col}", variable=switch_var)
            switch.grid(row=i, column=0, padx=10, pady=(0, 20))
            self.scrollable_frame_switches.append(switch)
            self.switch_variables.append(switch_var)

    def on_Barra_de_Carregamento(self):
        self.entry.configure(text="Lendo arquivo...")

        try:
            # Simulação de leitura do arquivo
            import time
            for i in range(100):
                time.sleep(0.1)  # Simulando uma operação demorada
                self.progress_bar['value'] = i + 1

                # Adicione as linhas abaixo para permitir o cancelamento
                if self._cancel_load:
                    self._cancel_load = False
                    self.progress_bar['value'] = 0
                    self.entry.configure(text="Carregamento cancelado.")
                    return

            self.progress_bar['value'] = 0  # Reinicia a barra de progresso

        except FileNotFoundError as e:
            self.entry.configure(text="Arquivo não encontrado.")
        except Exception as e:
            self.entry.configure(text="Erro ao carregar arquivo.")
    
    def cancel_load(self):
        # Este método será chamado quando o botão de cancelamento for pressionado
        self._cancel_load = True
        self.progress_bar['value'] = 0  # Reinicia a barra de progresso
        self.entry.configure(text="Carregamento cancelado.")

    def Ler_Arquivo_DataFrame(self):
        if self.selected_extension == ".csv":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo",
                                                    filetypes=(("Arquivos CSV", ".csv"), ("Todos os arquivos", ".*")))
            if file_path:  # Verifica se um arquivo foi selecionado
                self.on_Barra_de_Carregamento()

                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        if self.selected_codificacao == "iso":
                            df_temp = pd.read_csv(file, delimiter=self.determinar_delimitador(file_path), low_memory=False)

                            # # Converte todas as colunas para números se possível
                            # df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                            self.mostrar_colunas_disponiveis(df_temp)

                            # Armazena temporariamente o DataFrame no arquivo temporário
                            df_temp.to_csv(self.temp_file.name, index=False)

                            self.entry.configure(text="Arquivo carregado com sucesso! Visualize as colunas disponíveis.")
                        elif self.selected_codificacao == "utf-8":
                            df_temp = pd.read_csv(file, delimiter=self.determinar_delimitador(file_path), encoding='utf-8', low_memory=False)

                            # # Converte todas as colunas para números se possível
                            # df_temp = df_temp.apply(pd.to_numeric, errors='coerce')

                            self.mostrar_colunas_disponiveis(df_temp)

                            # Armazena temporariamente o DataFrame no arquivo temporário
                            df_temp.to_csv(self.temp_file.name, index=False)

                            self.entry.configure(text="Arquivo carregado com sucesso! Visualize as colunas disponíveis.")
                        else:
                            self.entry.configure(text="Nenhum tipo de codificação selecionado")

                except UnicodeDecodeError as e:
                    self.entry.configure(text=f"Erro ao ler o arquivo: {str(e)}. Certifique-se de escolher a codificação correta.")
                except Exception as e:
                    self.entry.configure(text=f"Erro ao carregar arquivo: {str(e)}")
            else:
                self.entry.configure(text="Nenhum arquivo selecionado.")
        elif self.selected_extension == ".xls":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo Excel",
                                                    filetypes=(("Arquivos Excel", ".xls .xlsx"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o arquivo Excel usando o pandas
                    df_temp = pd.read_excel(file_path, low_memory=False)

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo Excel carregado com sucesso! Visualize as colunas disponíveis.")

                elif self.selected_codificacao == "utf-8":
                    df_temp = pd.read_excel(file_path, low_memory=False, encoding='utf-8')
                
                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo Excel carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo Excel selecionado.")

        elif self.selected_extension == ".json":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo JSON",
                                                    filetypes=(("Arquivos JSON", ".json"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o arquivo JSON usando o pandas
                    df_temp = pd.read_json(file_path, orient='records', lines=True, encoding='latin-1')

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo JSON carregado com sucesso! Visualize as colunas disponíveis.")

                elif self.selected_codificacao == "utf-8":
                    # Lê o arquivo JSON usando o pandas
                    df_temp = pd.read_json(file_path, orient='records', lines=True, encoding='utf-8')
                
                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo JSON carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo JSON selecionado.")

        elif self.selected_extension == ".dbf":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo DBF",
                                                    filetypes=(("Arquivos DBF", ".dbf"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                # if self.selected_codificacao == "iso":
                # Usa a função dbf2DF para ler o arquivo DBF
                df_temp = self.dbf2DF(file_path)
                # elif self.selected_codificacao == "utf-8":
                #     # Supondo que a função dbf2DF aceita um parâmetro de codificação
                #     df_temp = self.dbf2DF(file_path, encoding='utf-8')
                # else:
                #     self.entry.configure(text="Nenhun tipo de codificação selecionado")

                if self.selected_codificacao == "iso":
                    df_temp.to_csv(self.temp_file.name, index=False)

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    self.entry.configure(text="Arquivo DBF carregado com sucesso! Visualize as colunas disponíveis.")
                elif self.selected_codificacao == "utf-8":
                    # Exemplo: Salva o DataFrame em um arquivo CSV com encoding 'utf-8'
                    df_temp.to_csv(self.temp_file.name, index=False, encoding='latin-1')

                    # Converte todas as colunas para números se possível
                    df_temp = df_temp.apply(pd.to_numeric, errors='ignore')

                    self.mostrar_colunas_disponiveis(df_temp)

                    self.entry.configure(text="Arquivo DBF carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhun tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo DBF selecionado.")

        elif self.selected_extension == ".txt":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo de texto",
                                                    filetypes=(("Arquivos de Texto", ".txt"), ("Todos os arquivos", ".*")))
            if file_path:
                self.on_Barra_de_Carregamento()

                if self.selected_codificacao == "iso":
                    # Lê o conteúdo do arquivo de texto usando a função open() com encoding 'latin-1'
                    with open(file_path, 'r', encoding='latin-1') as file:
                        # Lê todas as linhas do arquivo e remove os caracteres de quebra de linha
                        lines = [line.strip() for line in file.readlines()]

                        # Separa cada linha em colunas usando um delimitador (por exemplo, tabulação)
                        # Você precisa substituir '\t' pelo delimitador correto usado no seu arquivo de texto
                        df_temp = pd.DataFrame([line.split('\t') for line in lines])

                        # Define o cabeçalho do DataFrame como a primeira linha do arquivo de texto
                        df_temp.columns = df_temp.iloc[0]
                        df_temp = df_temp[1:]  # Remove a primeira linha (cabeçalho) do DataFrame

                    self.mostrar_colunas_disponiveis(df_temp)
                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo de texto carregado com sucesso! Visualize as colunas disponíveis.")
                elif self.selected_codificacao == "utf-8":
                    # Lê o conteúdo do arquivo de texto usando a função open() com encoding 'utf-8'
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = [line.strip() for line in file.readlines()]
                        df_temp = pd.DataFrame([line.split('\t') for line in lines])
                        df_temp.columns = df_temp.iloc[0]
                        df_temp = df_temp[1:]

                    self.mostrar_colunas_disponiveis(df_temp)
                    df_temp.to_csv(self.temp_file.name, index=False)

                    self.entry.configure(text="Arquivo de texto carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    self.entry.configure(text="Nenhum tipo de codificação selecionado")
            else:
                self.entry.configure(text="Nenhum arquivo de texto selecionado.")

        else:
            self.entry.configure(text="Selecione um arquivo CSV, Excel, JSON, DBF ou de texto antes de ler!")

    def carregar_subtabela(self):
        # Carregar o segundo arquivo temporário como sub tabela
        df_sub_tabela = pd.read_csv(self.temp_file.name, index=False)

        # Realizar a operação Table.merge() entre a tabela principal e a sub tabela
        if self.checkbox_JOIN.get():
            tabela_mesclada = self.tabela_dataframe.merge(df_sub_tabela, how='inner', on='coluna_de_juncao')
            self.mostrar_tabela(tabela_mesclada)
        else:
            for i, df in enumerate(self.dataframes_carregados, start=1):
                self.mostrar_dataframe_selecionado(df, f"DataFrame {i} - SUBTABLE")

    def carregar_colunas_selecionadas(self):
        self.on_Barra_de_Carregamento()
        if not os.path.exists(self.temp_file.name):
            self.entry.configure(text="Nenhum arquivo temporário encontrado. Carregue um arquivo antes de selecionar colunas.")
            return

        # Obtém as colunas selecionadas
        colunas_selecionadas = [self.scrollable_frame_switches[i].cget("text") for i, var in enumerate(self.switch_variables) if var.get()]
        
        if not self.checkbox_JOIN.winfo_ismapped():  # Verifica se o checkbox_JOIN está oculto
            if colunas_selecionadas:
                while True:
                    # Carrega apenas as colunas selecionadas do arquivo temporário
                    df_selecionado = pd.read_csv(self.temp_file.name, usecols=colunas_selecionadas, low_memory=False)

                    resposta = tkinter.messagebox.askquestion("SAUDADOS", "Deseja carregar mais alguma base de dados?")

                    self.dataframes_carregados.append(df_selecionado)

                    nome_arquivo = os.path.basename(self.selected_file)

                    if resposta == 'yes':
                        # Atualiza a interface
                        self.entry.configure(text="Aguardando a leitura do próximo arquivo...")
                        self.checkbox_JOIN.grid()

                        novo_temp_file = tempfile.NamedTemporaryFile(delete=False)
                        df_selecionado.to_csv(novo_temp_file.name, index=False)
                        self.temp_file.close()
                        self.temp_file = novo_temp_file

                        # self.scrollable_frame.destroy()
                        # self.scrollable_frame_switches = []
                        # self.switch_variables = []
                        # # self.init_interface()
                        self.entry.configure(text=f"Arquivo '{nome_arquivo}' carregado com sucesso! Visualize as colunas disponíveis.")
                    else:
                        for i, df in enumerate(self.dataframes_carregados, start=1):
                            self.mostrar_dataframe_selecionado(df, f"DataFrame {i} - {nome_arquivo}")

                    break
            else:
                self.entry.configure(text="Selecione pelo menos uma coluna para carregar.")
        else:
            self.carregar_subtabela()
        #     join_types = ["inner", "outer", "left", "right"]
        #     join_type = tkinter.messagebox.askquestion("Selecionar Tipo de Join", "Selecione o tipo de join:",
        #                                             icon='question', type='yesno', default='yes')

        #     # Realize o join entre os DataFrames
        #     df_original = pd.read_csv(self.temp_file.name, low_memory=False)
        #     # Substitua 'coluna_de_juncao' pelo nome real da coluna de junção
        #     df_final = pd.merge(df_original, df_selecionado, on=f"DataFrame {i} - {nome_arquivo}", how=join_types)

        #     # Agora 'df_final' contém o resultado do join
        #     self.mostrar_dataframe_selecionado(df_final, "Resultado do Join")                            

    def aplicar_filtro(self, dataframe, expressao_filtro, text_widget):
        try:
            # Aplica o filtro ao DataFrame
            dataframe_filtrado = dataframe.query(expressao_filtro)

            # Atualiza o texto na janela com o DataFrame filtrado
            text_widget.delete("1.0", tkinter.END)  # Limpa o conteúdo atual do Text widget
            text_widget.insert("0.0", dataframe_filtrado.to_string())

        except Exception as e:
            # Exibe uma mensagem de erro se a expressão de filtro for inválida
            tkinter.messagebox.showerror("Erro", f"Erro ao aplicar filtro: {str(e)}")

    def mostrar_dataframe_selecionado(self, dataframe, nome_arquivo):
        # Cria uma nova janela para exibir o DataFrame
        janela_dataframe = customtkinter.CTkToplevel(self)
        janela_dataframe.title(f"DataFrame - {nome_arquivo}")
        janela_dataframe.geometry(f"{1100}x{580}")
        #janela_dataframe.resizable(width=False, height=False)

        # Divide a janela em duas colunas
        janela_dataframe.grid_columnconfigure(0, weight=1)  # Coluna 0
        janela_dataframe.grid_columnconfigure(1, weight=3)  # Coluna 1 (larger)

        # Divide a janela em duas linhas
        janela_dataframe.grid_rowconfigure(0, weight=1)  # Linha 0
        janela_dataframe.grid_rowconfigure(1, weight=3)  # Linha 1 (larger)

        # Parte menor
        self.parte_menor = customtkinter.CTkFrame(janela_dataframe, width=200, corner_radius=0)
        self.parte_menor.grid(row=0, column=0, rowspan=2, sticky="nsew", padx=(0, 10))  # Adiciona padx para criar o espaçamento
        self.parte_menor.grid_rowconfigure(4, weight=1)

        # self.Inicio = customtkinter.CTkFrame(self.parte_menor, width=200, corner_radius=0)  # Novo subframe
        # self.Inicio.grid(row=0, column=0, sticky="nsew")  # Usando grid no subframe

        #mudar tema
        self.appearance_mode_label2 = customtkinter.CTkLabel(self.parte_menor, text="Aparência:", anchor="w")
        self.appearance_mode_label2.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu2 = customtkinter.CTkOptionMenu(self.parte_menor, values=["Light", "Dark", "System"],
                                                                    command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu2.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label2 = customtkinter.CTkLabel(self.parte_menor, text="Escala:", anchor="w")
        self.scaling_label2.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu2 = customtkinter.CTkOptionMenu(self.parte_menor, values=["80%", "90%", "100%", "110%", "120%"],
                                                            command=self.change_scaling_event)
        self.scaling_optionemenu2.grid(row=8, column=0, padx=20, pady=(10, 20))

        # Parte maior
        self.parte_maior = customtkinter.CTkFrame(janela_dataframe, width=200, corner_radius=0)
        self.parte_maior.grid(row=0, column=1, rowspan=2, sticky="nsew", padx=(10, 0))  # Adiciona padx para criar o espaçamento
        self.parte_maior.grid_rowconfigure(4, weight=1)

        # Adiciona a tabela como um widget
        self.tabela_dataframe = DataFrameTable(self.parte_maior, dataframe=dataframe, showtoolbar=True, showstatusbar=True, editable=False)
        self.tabela_dataframe.grid(row=0, column=0, columnspan=3, padx=20, pady=20)

        try:
            # Seu código para mostrar a tabela
            self.tabela_dataframe.show()
        except Exception as e:
            print(f"Erro ao mostrar a tabela: {e}")

        # Atualiza a interface
        janela_dataframe.update_idletasks()

if __name__ == "__main__":
    app = AppSaudados()
    app.mainloop()